In [1]:
# Imports
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import asyncio
import os
from pydantic import BaseModel
from sendgrid.helpers.mail import Mail, From, To, Content

In [82]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyBM
Groq API Key exists and begins gsk_UFJY


In [6]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
#DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [15]:
# Create 3 System Instructions for 3 different sales agent
instruction1 = f""" You are a professional sales agent working for OOIMA FAASHION, a company that deals in apparel business
especially custom products like printed t-shirts, caps, mouse pads, mugs. Your job is to write professional and serious
cold emails. 
Note: Do not provide subject for the mail
"""
instruction2 = f""" You are a humorous, engaging sales agent working for OOIMA FAASHION, a company that deals in apparel business
especially custom products like printed t-shirts, caps, mouse pads, mugs. Your job is to write witty, engaging
cold emails that are likely to get a response. 
Note: Do not provide subject for the mail
"""
instruction3 = f""" You are a busy sales agent working for OOIMA FAASHION, a company that deals in apparel business
especially custom products like printed t-shirts, caps, mouse pads, mugs. Your job is to concise, to the point cold emails. 
Note: Do not provide subject for the mail 
"""

In [7]:
# Create different models
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

gemini_model= OpenAIChatCompletionsModel(openai_client=gemini_client, model="gemini-2.5-flash")
groq_model = OpenAIChatCompletionsModel(openai_client=groq_client, model="llama-3.3-70b-versatile")


In [8]:
openai_sales_agent = Agent(name="OpenAI Sales Agent", instructions=instruction1, model="gpt-4o-mini")
gemini_sales_agent = Agent(name="Gemini Sales Agent", instructions= instruction2, model=gemini_model)
groq_sales_agent = Agent(name="Groq sales agent", instructions=instruction3, model=groq_model)


In [ ]:
description= "Write a cold sales email"
responses = await asyncio.gather(
    Runner.run(openai_sales_agent,description),
    Runner.run(gemini_sales_agent, description),
    Runner.run(groq_sales_agent, description)
)
for response in responses:
    print(f"{response.last_agent.name} \n{response.final_output} \n")
    #print(response.final_output)

OpenAI Sales Agent 
 Dear [Recipient's Name],

I hope this message finds you well.

My name is [Your Name], and I represent OOIMA FAASHION, a leading provider of custom apparel and promotional products. We specialize in crafting high-quality printed t-shirts, caps, mouse pads, and mugs tailored to our clients' unique branding needs.

At OOIMA FAASHION, we understand the importance of maintaining a strong brand identity. Our customizable products not only serve as excellent promotional tools but also create lasting impressions on your audience. Whether you are looking to enhance employee uniforms, create branded merchandise for events, or establish a unique presence online, we can support your vision with our wide range of options.

We pride ourselves on our dedication to quality and customer satisfaction. Our design team works closely with clients to ensure that every product reflects their brand accurately and compellingly.

I would love the opportunity to discuss how we can partner t

In [31]:
# convert these agent as tool
description = "Please write a cold sales email"
tool1 = openai_sales_agent.as_tool(tool_name="Professional_Mail_Drafting_tool_using_openai", tool_description=description)
tool2 = gemini_sales_agent.as_tool(tool_name="Humorous_Mail_Drafting_tool_using_gemini", tool_description=description)
tool3 = groq_sales_agent.as_tool(tool_name="Busy_Mail_Drafting_tool_using_Groq", tool_description=description)

In [19]:
# Create a function tool to send emails using SendGrid


@function_tool
def send_html_email(subject:str, html_body:str)-> dict[str, str]:
    """Send a sales email with the given subject and HTML body to the sales prospect"""
    sg = sendgrid.SendGridAPIClient(os.getenv("SENDGRID_API_KEY"))
    message = Mail(from_email="sumanprakash.nitp@gmail.com",
    to_emails="bittu1suman2@gmail.com",
    html_content=html_body,
    subject=subject)
    sg.send(message=message)
    return {"Status":"Success"}

In [20]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [22]:
email_tools = [subject_tool, html_tool, send_html_email]

In [23]:
# Create an emailer agent to send html email to sales prospect using email_tools
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [33]:
# define the tools and handoffs to be used by sales manager agent
mail_drafting_tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [34]:
sales_manager_instructions = """
You are a Sales Manager at OOIMA FASHION. Your goal is to find the single best cold sales email using the tools in mail_drafting_tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three tools in mail_drafting_tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the emailer_agent. The emailer_agent will take care of formatting and sending.
 
Crucial Rules:
- You must use the tools in mail_drafting_tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the emailer_agent — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=mail_drafting_tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [52]:
# implement input guardrails
class NameCheckOutput(BaseModel):
    is_name_in_message:bool
    name:str

guardrail_agent = Agent(
    name="Name Check",
    instructions="check if the user includes someone's personal name in the task they want you do",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)


In [53]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result =await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found name":result.final_output.name}, tripwire_triggered=is_name_in_message)

In [54]:
careful_sales_manager = Agent(name="Careful Sales Manager",
instructions=sales_manager_instructions,
handoffs=handoffs,
tools=mail_drafting_tools,
model="gpt-4o-mini",
input_guardrails=[guardrail_against_name])

In [55]:
message = "Please send a cold sales email to 'Dear CEO' from Alice"
with trace("Automated Sales BDR"):
    await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [56]:
message = "Please send a cold sales email to 'Dear CEO' from 'Head of Marketing'"
with trace("Automated Sales BDR"):
    await Runner.run(careful_sales_manager, message)

In [76]:
class MailCheckOutput(BaseModel):
    is_email_valid: bool
    sender_name: str
    signature: str



In [77]:
output_guardrail_instructions = f"""Check if the final email to be sent to the customer is appropriate\n
- Make sure there is a proper name of sender is present.
- Make sure the email signature contains valid and all fields populated."""
guardrail_agent_output = Agent(
    name="Email Check Guardrail",
    instructions= output_guardrail_instructions,
    output_type=MailCheckOutput,
    model=gemini_model
)

In [78]:
from agents import output_guardrail
@output_guardrail
async def guardrail_final_email(ctx, agent, final_email):
    result = await Runner.run(guardrail_agent_output, final_email, context=ctx.context)
    is_email_valid = result.final_output.is_email_valid
    return GuardrailFunctionOutput(output_info={"Invalid Email":result.final_output}, tripwire_triggered=not(is_email_valid))

In [81]:
guardrail_final_email

OutputGuardrail(guardrail_function=<function guardrail_final_email at 0x000001972DA8BC40>, name='guardrail_final_email')

In [79]:
super_sales_manager = Agent(name="Super Sales Manager",
instructions=sales_manager_instructions,
handoffs=handoffs,
tools=mail_drafting_tools,
model="gpt-4o-mini",
output_guardrails=[guardrail_final_email])

In [80]:
message = "Please send a cold sales email to 'Dear CEO'"
with trace("Automated Sales BDR"):
    await Runner.run(super_sales_manager, message)